### IMPORTANDO MÓDULOS

In [1]:
from pandas import DataFrame, read_csv
import numpy as np
from sklearn.preprocessing import StandardScaler

### CARREGANDO DADOS TREINO

In [7]:
dataset_treino: DataFrame = read_csv(
    ".\\T02_ClassicarRegredir\\treino_sinais_vitais_com_label.csv",
    sep = ',',
    index_col = 'index'
)

# Parâmetros previsores/independentes
x_dataset = dataset_treino.drop(columns = ['pressao_sistolica', 'pressao_diastolica', 'gravidade', 'classe'], axis = 1)

# Parâmetros de classe/dependentes
y_dataset_classes = dataset_treino['classe']

# Parâmetros de dependentes
y_dataset_regressao = dataset_treino['gravidade']

x_dataset, y_dataset_classes, y_dataset_regressao

(       qualidade_pressao     pulso  respiracao
 index                                         
 1                -8.4577   56.8384      9.2229
 2                -8.6134  146.6000     14.6048
 3                 8.7333  114.0737      4.2997
 4                 8.6493  124.7851      3.8199
 5                 8.6096   51.3438     21.0420
 ...                  ...       ...         ...
 796              -8.4813  185.5561      8.8162
 797               4.8693  198.0136     10.8730
 798               4.7972  184.0678      8.9774
 799               8.2539   86.2256     18.9484
 800              -0.2918  155.5342     19.9939
 
 [800 rows x 3 columns],
 index
 1      2
 2      3
 3      2
 4      1
 5      2
       ..
 796    1
 797    2
 798    1
 799    3
 800    2
 Name: classe, Length: 800, dtype: int64,
 index
 1      33.5156
 2      68.3047
 3      30.3124
 4      24.0960
 5      30.8344
         ...   
 796    13.3046
 797    34.2401
 798    24.3470
 799    53.4373
 800    39.4575
 Name: 

PADRONIZAÇÃO DOS DADOS DE TREINO

In [8]:
padronizador  = StandardScaler()

# Transformação dos dados e fit do padronizador de acordo com estes dados
x_dataset_padronizado = padronizador.fit_transform(x_dataset)
x_dataset_padronizado

array([[-1.32822746, -0.80499737, -0.3133257 ],
       [-1.35030829,  0.7469566 ,  0.55705413],
       [ 1.10974081,  0.1845856 , -1.10952296],
       ...,
       [ 0.5515367 ,  1.39476481, -0.35302883],
       [ 1.04175395, -0.29690057,  1.25951643],
       [-0.17016771,  0.90142652,  1.42859837]])

### MODELAGEM CLASSIFICAÇÃO

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_validate

#### ÁRVORE DE DECISÃO

INSTANCIANDO O MODELO

In [37]:
arvore_decisao_1 = DecisionTreeClassifier(
    ccp_alpha = 0.0,
    class_weight = None,
    criterion = 'entropy',
    max_depth = 60,
    max_features = None,
    max_leaf_nodes = None,
    min_impurity_decrease = 0.0,
    min_samples_leaf = 4,
    min_samples_split = 4,
    min_weight_fraction_leaf = 0.0,
    random_state = 493827156,
    splitter = 'best'
)

arvore_decisao_2 = DecisionTreeClassifier(
    ccp_alpha = 0.0,
    class_weight = None,
    criterion = 'gini',
    max_depth = 208,
    max_features = None,
    max_leaf_nodes = None,
    min_impurity_decrease = 0.0,
    min_samples_leaf = 10,
    min_samples_split = 40,
    min_weight_fraction_leaf = 0.0,
    random_state = 370370367,
    splitter = 'best'
)


TREINO

In [46]:
metricas1 = {}
metricas2 = {}

# for n_split in range(2, 21):
resultado = cross_validate(
        arvore_decisao_1,
        x_dataset_padronizado,
        y_dataset_classes,
        scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'],
        n_jobs = -1,
        cv = KFold(
            n_splits = 7,
            # n_splits = 16,
            shuffle = True,
            random_state = 42,
        ),
    )

metricas1['acuracia'] = resultado['test_accuracy'].mean()
metricas1['precisao'] = resultado['test_precision_weighted'].mean()
metricas1['recall'] = resultado['test_recall_weighted'].mean()
metricas1['f1_score'] = resultado['test_f1_weighted'].mean()

# for n_split in range(2, 21):
resultado = cross_validate(
        arvore_decisao_2,
        x_dataset_padronizado,
        y_dataset_classes,
        scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'],
        n_jobs = -1,
        cv = KFold(
            n_splits = 7,
            # n_splits = 16,
            shuffle = True,
            random_state = 42,
        ),
    )

metricas2['acuracia'] = resultado['test_accuracy'].mean()
metricas2['precisao'] = resultado['test_precision_weighted'].mean()
metricas2['recall'] = resultado['test_recall_weighted'].mean()
metricas2['f1_score'] = resultado['test_f1_weighted'].mean()

print(f"{metricas1}")
print(f"{metricas2}")




{'acuracia': 0.8837310667974282, 'precisao': 0.8870572561208424, 'recall': 0.8837310667974282, 'f1_score': 0.8822655756834742}
{'acuracia': 0.8362318840579709, 'precisao': 0.8389740079507177, 'recall': 0.8362318840579709, 'f1_score': 0.8351695140746491}


FITTING

In [40]:
arvore_decisao_1.fit(x_dataset_padronizado, y_dataset_classes)

DecisionTreeClassifier(criterion='entropy', max_depth=60, min_samples_leaf=4,
                       min_samples_split=4, random_state=493827156)

#### FUZZY

### MODELAGEM REGRESSÃO

In [10]:
from sklearn.neural_network import MLPRegressor

#### REDE NEURAL

INSTANCIANDO O MODELO

In [13]:
rede_neural = MLPRegressor(
    activation = 'relu',
    alpha = 0.0001,
    batch_size = 'auto',
    beta_1 = 0.9,
    beta_2 = 0.999,
    early_stopping = True,
    epsilon = 1e-08,
    hidden_layer_sizes = [14, 31, 31, 31, 31, 31, 31, 31, 14],
    learning_rate = 'adaptive',
    learning_rate_init = 0.001,
    max_fun = 15000,
    max_iter = 10000,
    momentum = 0.9,
    n_iter_no_change = 10,
    nesterovs_momentum = True,
    power_t = 0.5,
    random_state = 370370367,
    shuffle = False,
    solver = 'adam',
    tol = 0.0001,
    validation_fraction = 0.1,
    verbose = False,
    warm_start = False
)

rede_neural_2 = MLPRegressor(
    activation = 'relu',
    alpha = 0.0001,
    batch_size = 'auto',
    beta_1 = 0.9,
    beta_2 = 0.999,
    early_stopping = True,
    epsilon = 1e-08,
    hidden_layer_sizes = [12, 32, 32, 12],
    learning_rate = 'adaptive',
    learning_rate_init = 0.001,
    max_fun = 15000,
    max_iter = 10000,
    momentum = 0.9,
    n_iter_no_change = 10,
    nesterovs_momentum = True,
    power_t = 0.5,
    random_state = 370370367,
    shuffle = False,
    solver = 'sgd',
    tol = 0.0001,
    validation_fraction = 0.1,
    verbose = False,
    warm_start = False
)

TREINO

In [14]:
# for n_split in range(2, 20):
resultado = cross_validate(
    rede_neural_1,
    x_dataset_padronizado,
    y_dataset_regressao,
    scoring = ['r2', 'neg_mean_absolute_error'],
    n_jobs = -1,
    cv = KFold(
        n_splits = 16,
        # n_splits = 11,
        shuffle = True,
        random_state = 42,
    ),
    # error_score='raise'
)

print(f"R2: {np.mean(resultado['test_r2'])}; MAE = {np.mean(resultado['test_neg_mean_absolute_error'])*(-1)}")

R2: 0.9675985788385831; MAE = 1.9753073011918834


FITTING

In [15]:
rede_neural_1.fit(x_dataset_padronizado, y_dataset_regressao)

MLPRegressor(early_stopping=True,
             hidden_layer_sizes=[14, 31, 31, 31, 31, 31, 31, 31, 14],
             learning_rate='adaptive', max_iter=10000, random_state=370370367,
             shuffle=False)

### CARREGANDO DADOS TESTE AULA

CARREGANDO OS DADOS

In [16]:
# COLOCAR A SEPARAÇÃO E IMPORTAÇÃO DOS DADOS AQUI.
dataset_teste_aula: DataFrame = read_csv(
    ".\\T02_ClassicarRegredir\\teste_cego.csv",
    sep = ',',
    index_col = 'index'
)

# Parâmetros previsores/independentes
# x_dataset_aula = dataset_teste_aula.drop(columns = ['gravidade', 'classe'], axis = 1)
dataset_teste_aula
# Parâmetros de classe/dependentes
# y_dataset_aula_classes = dataset_teste_aula['classe']

# Parâmetros de dependentes
# y_dataset_aula_regressao = dataset_teste_aula['gravidade']

,qualidade_pressao,pulso,respiracao
index,,,
1,8.665540,24.786456,6.983413
2,8.733333,98.071459,17.919875
3,0.000000,170.599631,17.359617
4,8.733333,174.785481,18.749806
5,-0.000000,54.058866,11.124006
...,...,...,...
326,4.271582,143.042503,20.101657
327,4.666667,128.412166,12.282268
328,3.777818,83.809659,13.175098


PADRONIZANDO O DATASET DE TESTE

In [17]:
x_dataset_aula_padronizado = padronizador.transform(dataset_teste_aula)
x_dataset_aula_padronizado

array([[ 1.10013132e+00, -1.35916688e+00, -6.75503432e-01],
       [ 1.10974549e+00, -9.20888472e-02,  1.09317974e+00],
       [-1.28785645e-01,  1.16190378e+00,  1.00257284e+00],
       [ 1.10974549e+00,  1.23427600e+00,  1.22739911e+00],
       [-1.28785645e-01, -8.53054767e-01, -5.87214073e-03],
       [-1.28785645e-01, -1.06686986e+00,  4.56745277e-01],
       [-6.06056897e-01,  1.65950974e-01,  1.57831829e+00],
       [-1.28785645e-01,  4.26445868e-01, -2.25297810e-01],
       [ 1.10974549e+00, -3.45654328e-01, -1.59134359e+00],
       [ 1.10974549e+00, -1.07546578e+00,  1.27893910e+00],
       [-7.43976467e-01,  1.49018379e+00,  4.40831844e-01],
       [ 1.10974549e+00, -1.50392023e+00, -5.41562878e-01],
       [ 1.09422508e+00, -1.42218053e+00,  1.74236983e+00],
       [ 5.33024956e-01, -1.29654748e+00, -1.00730188e+00],
       [-1.28615465e-01, -2.66098585e-01,  1.71024685e+00],
       [-7.87278732e-01,  3.59523610e-01,  3.47621197e-01],
       [ 5.26657676e-01,  1.96139520e-01

In [41]:
predicoes_arvore = arvore_decisao_1.predict(x_dataset_aula_padronizado)
predicoes_rede = rede_neural_1.predict(x_dataset_aula_padronizado)

In [43]:
for aux in predicoes_arvore:
    print(aux)

1
3
3
2
2
2
2
2
2
2
3
1
1
1
3
3
2
1
1
1
2
2
3
2
2
2
1
2
1
3
1
2
2
3
2
2
3
2
2
2
2
2
2
3
2
2
2
3
2
2
2
2
1
1
3
3
2
2
2
2
1
2
2
2
3
2
2
2
2
3
1
1
1
3
2
2
2
3
2
2
2
2
2
2
2
2
1
2
2
2
2
2
3
3
2
2
2
1
3
2
2
1
2
2
1
2
2
1
2
3
3
2
2
2
2
2
2
1
3
3
2
2
3
1
2
2
2
2
2
1
2
2
2
1
2
2
1
1
2
3
1
1
2
2
2
2
3
2
2
2
2
3
1
2
2
2
3
2
2
2
2
3
2
2
2
3
3
2
3
3
3
1
2
1
2
3
2
1
3
1
1
3
2
2
2
3
2
2
2
3
1
1
2
2
2
3
2
2
2
1
2
3
2
3
1
3
1
2
1
2
2
2
3
2
1
2
2
2
2
2
2
2
1
2
3
2
3
3
3
2
2
2
2
1
2
2
2
2
2
2
2
2
2
3
2
2
3
3
3
1
1
2
1
3
3
3
2
2
2
2
2
2
2
3
3
2
2
2
3
3
2
2
2
2
1
1
2
3
2
2
2
2
2
2
1
2
2
1
2
2
2
3
2
3
2
2
2
2
2
2
2
2
2
3
3
2
3
3
2
2
1
2
1
2
1
3
3
2
3
2
1
3
1
2
2
2
2
3
2
2


In [25]:
for aux in predicoes_rede:
    print(aux)

16.009381091442926
52.96436272510358
60.37391373195194
27.03283779139173
56.18521502063665
63.009433571343834
53.90013563317417
40.90187702017903
39.89013161841607
34.64179628992221
62.203513376593534
14.10596305646443
12.555604251169019
26.58840301560983
68.56817481141796
61.215547877680436
38.15333854295267
13.179999858872964
13.920698167511134
16.373905767894684
34.261068194353705
45.09265194859855
68.17779184165985
28.56200316314135
48.47770072958269
34.202527717275174
20.91323511169591
65.11172828580297
28.19496602188826
69.34281029035063
28.631220622593744
21.790165412009635
35.60794193566561
60.70686079586303
32.678725142917685
39.02969480040654
62.725672751391784
31.230023365048
36.50717619668148
22.46666607958342
58.7795439795248
37.238678315732
52.62606356440671
56.388732437667684
43.83127340006354
57.000882949980614
27.89105484152912
70.5658215228163
38.83051664484
29.472551214036834
26.319789624538327
43.419178957232106
26.517679504039698
18.481150677785184
61.2094971015388